In [1]:
from climakitae.core.data_interface import (
    get_data_options, 
    get_subsetting_options, 
    get_data
)
# import climakitae as ck

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.backends.backend_pdf import PdfPages
import time
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point
import contextily as cx


print('done')

done


In [ ]:
def calculate_and_plot(ds, variable, warming_levels,timescale, percentiles):
    print(f"Starting calculations for {variable}...")
    print(ds)
    start_time = time.time()
    
    if timescale == "monthly":
        # Define the base month and center point (e.g., January 2000)
        base_month = 1  # January

        # Create a new coordinate for month
        months = (base_month + ds['time_delta'].values) % 12
        months[months == 0] = 12  # Ensure months are in the range [1, 12]
        ds = ds.assign_coords(month=('time_delta', months))

        # Group by the new month coordinate and calculate the mean
        print(f"Calculating monthly climatology for {variable}...")
        climatology_start_time = time.time()
        climatology = ds.groupby('month').mean(dim='time_delta', skipna=True)
        
        if variable == "Precipitation (total)":
            days_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) 

            # Convert monthly total precipitation to daily average within each month
            climatology = climatology / xr.DataArray(days_in_month, dims=["month"])
            climatology.attrs['extended_description'] = 'average daily precip'
        
    if timescale == "hourly":
        hours_in_year = 365 * 24
        # Calculate the hour of the year
        hour_of_year = ds['time_delta'].values % hours_in_year
        # Assign the new hour_of_year coordinate to the dataset
        ds = ds.assign_coords(hour_of_year=('time_delta', hour_of_year))

        hours_in_month = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) * 24
        cumulative_hours = np.cumsum(hours_in_month)

        # Function to map hour_of_year to month
        def hour_to_month(hour):
            return np.searchsorted(cumulative_hours, hour) + 1

        # Apply the function to calculate the month for each hour_of_year
        months = hour_to_month(ds['hour_of_year'].values)
        ds = ds.assign_coords(month=('time_delta', months))

        # Group by the new month coordinate and calculate the mean
        print(f"Calculating hourly climatology for {variable}...")
        climatology_start_time = time.time()
        climatology = ds.groupby('month').max(dim='time_delta', skipna=True)
        
    time_treatment = "avg."
        
    climatology.load()
    print(climatology)


    print(f"Monthly climatology for {variable} calculated in {time.time() - climatology_start_time:.2f} seconds.")
    
    # Calculate the specified percentiles
    print(f"Calculating percentiles for time series {variable}...")
    percentiles_start_time = time.time()
    percentile_values_ts = {p: ds.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}
    print(f"Percentiles for {variable} calculated in {time.time() - percentiles_start_time:.2f} seconds.")

    # Calculate the specified percentiles
    print(f"Calculating percentiles for climo {variable}...")
    percentiles_start_time = time.time()
    percentile_values = {p: climatology.quantile(p / 100.0, dim='simulation', skipna=True) for p in percentiles}
    print(f"Percentiles for {variable} calculated in {time.time() - percentiles_start_time:.2f} seconds.")

    lon2d, lat2d = np.meshgrid(climatology.lon, climatology.lat)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    x_web, y_web = transformer.transform(lon2d, lat2d)
    transformer_back = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    lon_back, lat_back = transformer_back.transform(x_web, y_web) 

    # Define the x-axis values (months)
    months = np.arange(1, 13)
    
    gwl_years = {
    1.0: 2010,
    1.5: 2030,
    2.0: 2050
    }
    
    print(percentile_values_ts)
    # del climatology
    # Create a PDF file to save the figures
    with PdfPages(f'{variable}_climatology.pdf') as pdf:
        print(f"Generating full time series plots for {variable}...")
        time_series_plots_start_time = time.time()
        
        # plot full time series 
        if len(warming_levels) == 1:
            fig, axs = plt.subplots(1, 1, figsize=(12, 12))
            axs = [axs]  # Make it a list to keep the indexing consistent
        else:
            fig, axs = plt.subplots(len(warming_levels), 1, figsize=(12, 12))
        fig.suptitle(f'{variable} ', fontsize=16)

        for i, level in enumerate(warming_levels):
            year = gwl_years[level]

            axs[i].plot(ds['time_delta'].values, percentile_values_ts[percentiles[0]].mean(dim='lat').mean(dim='lon').sel(warming_level=level), linestyle='--', color='grey', label=f'{percentiles[0]}th Percentile')

            axs[i].plot(ds['time_delta'].values, percentile_values_ts[percentiles[1]].mean(dim='lat').mean(dim='lon').sel(warming_level=level), linestyle='-.', color='grey', label=f'{percentiles[1]}th Percentile')

            axs[i].plot(ds['time_delta'].values, percentile_values_ts[percentiles[2]].mean(dim='lat').mean(dim='lon').sel(warming_level=level), linestyle='-.', color='grey', label=f'{percentiles[2]}th Percentile')
            # Shading between the 10th and 90th percentiles
            axs[i].fill_between(
                ds['time_delta'].values,
                percentile_values_ts[percentiles[0]].mean(dim='lat').mean(dim='lon').sel(warming_level=level),
                percentile_values_ts[percentiles[2]].mean(dim='lat').mean(dim='lon').sel(warming_level=level),
                alpha=0.3
            )

            axs[i].set_xlabel('Month')
            axs[i].set_ylabel(ds.attrs['units'])
            axs[i].set_title(f'{year} Time Series (GWL {level}) {ds.attrs['extended_description']}')
            axs[i].legend()

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close(fig)
        print(f"Time series plots for {variable} generated in {time.time() - time_series_plots_start_time:.2f} seconds.")
            
        del percentile_values_ts
        
        if len(warming_levels) == 1:
            fig, axs = plt.subplots(1, 1, figsize=(12, 12))
            axs = [axs]  # Make it a list to keep the indexing consistent
        else:
            fig, axs = plt.subplots(len(warming_levels), 1, figsize=(12, 12))
        fig.suptitle(f'{variable} ', fontsize=16)

        for i, level in enumerate(warming_levels):
            year = gwl_years[level]

            axs[i].plot(months, percentile_values[percentiles[0]].mean(dim='lat').mean(dim='lon').sel(warming_level=level), linestyle='--', color='grey', label=f'{percentiles[0]}th Percentile')

            axs[i].plot(months, percentile_values[percentiles[1]].mean(dim='lat').mean(dim='lon').sel(warming_level=level), linestyle='-.', color='grey', label=f'{percentiles[1]}th Percentile')

            axs[i].plot(months, percentile_values[percentiles[2]].mean(dim='lat').mean(dim='lon').sel(warming_level=level), linestyle='-.', color='grey', label=f'{percentiles[2]}th Percentile')
            # Shading between the 10th and 90th percentiles
            axs[i].fill_between(
                months,
                percentile_values[percentiles[0]].mean(dim='lat').mean(dim='lon').sel(warming_level=level),
                percentile_values[percentiles[2]].mean(dim='lat').mean(dim='lon').sel(warming_level=level),
                alpha=0.3
            )

            axs[i].set_xlabel('Month')
            axs[i].set_ylabel(ds.attrs['units'])
            axs[i].set_title(f'{year} Time Series (GWL {level}) {climatology.attrs['extended_description']}')
            axs[i].legend()

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close(fig)
        print(f"Time series plots for {variable} generated in {time.time() - time_series_plots_start_time:.2f} seconds.")
        print(f"Generating spatial plots for {variable}...")
        
        if variable == "Precipitation (total)":
            cmap = 'YlGnBu'
        elif "temperature" in variable:
            cmap = 'YlOrRd'
        else:
            cmap = 'viridis'
        
        spatial_plots_start_time = time.time()
        for level in warming_levels:
            year = gwl_years[level]
            fig, axs = plt.subplots(3, 3, figsize=(20, 20))
            fig.suptitle(f'{variable}({climatology.attrs['extended_description']}) - {year} (GWL {level}) in SCE Service Territory', fontsize=16)

            # Mean
            # Plot lower percentile
            percentile_lower = percentile_values[percentiles[0]].sel(warming_level=level).mean(dim='month')
            ax = axs[0, 0]
            mesh = ax.pcolormesh(x_web, y_web, percentile_lower, cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[0]}th Percentile (temporal {time_treatment})')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Plot median
            ax = axs[0, 1]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[1]].sel(warming_level=level).mean(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[1]}th Percentile (temporal {time_treatment})')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Plot upper percentile
            ax = axs[0, 2]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[2]].sel(warming_level=level).mean(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[2]}th Percentile (temporal {time_treatment})')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Max
            # Plot lower percentile
            ax = axs[1, 0]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[0]].sel(warming_level=level).max(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[0]}th Percentile (temporal max.)')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Plot median
            ax = axs[1, 1]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[1]].sel(warming_level=level).max(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[1]}th Percentile (temporal max.)')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Plot upper percentile
            ax = axs[1, 2]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[2]].sel(warming_level=level).max(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[2]}th Percentile (temporal max.)')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            ### # Min
            # Plot lower percentile
            ax = axs[2, 0]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[0]].sel(warming_level=level).min(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[0]}th Percentile (temporal min.)')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Plot median
            ax = axs[2,1]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[1]].sel(warming_level=level).min(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[1]}th Percentile (temporal min.)')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")

            # Plot upper percentile
            ax = axs[2, 2]
            mesh = ax.pcolormesh(x_web, y_web, percentile_values[percentiles[2]].sel(warming_level=level).min(dim='month'), cmap=cmap)
            cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
            ax.set_title(f'{percentiles[2]}th Percentile (temporal min.)')
            ax.set_ylabel('Latitude')
            ax.set_xlabel('Longitude')
            plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
            # Set the tick labels and reduce the number of ticks by half
            ax.set_xticks(x_web[0, ::40])  # Adjust the step size to half
            ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
            ax.set_yticks(y_web[::40, 0])  # Adjust the step size to half
            ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
            # Rotate the tick labels
            plt.xticks(rotation=45)
            plt.yticks(rotation=45)
            # Label the axes
            plt.xlabel("Longitude")
            plt.ylabel("Latitude")


            plt.tight_layout(rect=[0, 0, 1, 0.96])
            pdf.savefig(fig)
            plt.close(fig)
        
        if len(warming_levels) > 1:
            print(f"Generating difference spatial plots for {variable}")
            difference_spatial_plots_start_time = time.time()
            for i in range(len(warming_levels) - 1):
                level_1 = warming_levels[0]
                level_2 = warming_levels[i + 1]

                fig_diff, axs_diff = plt.subplots(1, 3, figsize=(18, 6))
                fig_diff.suptitle(f'{variable} - Difference between GWL {level_2} and GWL {level_1} ({level_2}-{level_1})', fontsize=16)

                # Calculate differences
                diff_lower = (percentile_values[percentiles[0]].sel(warming_level=level_2).mean(dim='month') - 
                              percentile_values[percentiles[0]].sel(warming_level=level_1).mean(dim='month'))
                diff_median = (percentile_values[percentiles[1]].sel(warming_level=level_2).mean(dim='month') - 
                              percentile_values[percentiles[1]].sel(warming_level=level_1).mean(dim='month'))
                diff_upper = (percentile_values[percentiles[2]].sel(warming_level=level_2).mean(dim='month') - 
                              percentile_values[percentiles[2]].sel(warming_level=level_1).mean(dim='month'))
                
                # print(f"Saving difference NetCDF files for {variable}...")
                # diff_lower.to_netcdf(f'{variable}_diff_GWL{level_2}-GWL{level_1}_{percentiles[0]}th.nc')
                # diff_median.to_netcdf(f'{variable}_diff_GWL{level_2}-GWL{level_1}_{percentiles[1]}th.nc')
                # diff_upper.to_netcdf(f'{variable}_diff_GWL{level_2}-GWL{level_1}_{percentiles[2]}th.nc')

                # Find global min and max for colorbar limits
                vmin = min(diff_lower.min(), diff_median.min(), diff_upper.min())
                vmax = max(diff_lower.max(), diff_median.max(), diff_upper.max())
                norm = plt.Normalize(vmin=-vmax, vmax=vmax)

                # Plot differences
                for ax, diff, title in zip(axs_diff, [diff_lower, diff_median, diff_upper], 
                                           [f'Difference in {percentiles[0]}th Percentile (temporal avg.)', 
                                            f'Difference in {percentiles[1]}th Percentile (temporal avg.)', 
                                            f'Difference in {percentiles[2]}th Percentile (temporal avg.)']):
                    mesh = ax.pcolormesh(x_web, y_web, diff, cmap='coolwarm', norm=norm)
                    cx.add_basemap(ax, crs="EPSG:3857", attribution_size=2)
                    ax.set_title(title)
                    ax.set_ylabel('Latitude')
                    ax.set_xlabel('Longitude')
                    cbar = plt.colorbar(mesh, ax=ax, label=ds.attrs['units'])
                    cbar.set_label(ds.attrs['units'])
                    ax.set_xticks(x_web[0, ::40])
                    ax.set_xticklabels(np.round(lon_back[0, ::40], 2))
                    ax.set_yticks(y_web[::40, 0])
                    ax.set_yticklabels(np.round(lat_back[::40, 0], 2))
                    plt.xticks(rotation=45)
                    plt.yticks(rotation=45)

                plt.tight_layout(rect=[0, 0, 1, 0.96])
                pdf.savefig(fig_diff)
                plt.close(fig_diff)


        print(f"Difference spatial plots for {variable} generated in {time.time() - difference_spatial_plots_start_time:.2f} seconds.")

    print(f"Saving NetCDF files for {variable}...")
    netcdf_saving_start_time = time.time()
    for level in warming_levels:
        percentile_values[percentiles[0]].sel(warming_level=level).to_netcdf(f'{variable}_GWL{level}_{percentiles[0]}th.nc')
        percentile_values[percentiles[1]].sel(warming_level=level).to_netcdf(f'{variable}_GWL{level}_{percentiles[1]}th.nc')  
        percentile_values[percentiles[2]].sel(warming_level=level).to_netcdf(f'{variable}_GWL{level}_{percentiles[2]}th.nc')  
    print(f"NetCDF files for {variable} saved in {time.time() - netcdf_saving_start_time:.2f} seconds.")


    print(f"Finished processing {variable} in {time.time() - start_time:.2f} seconds.")
    

def process_variable(variable_unit):
    variable, unit,downscale,timescale,GWL = variable_unit
    print(f"Processing variable: {variable}")
    ds_start_time = time.time()
    ds = get_data(
        variable=variable,
        units=unit,
        downscaling_method=downscale,
        resolution="45 km",
        timescale=timescale,
        cached_area="Southern California Edison",
        approach="Warming Level",
        warming_level_window=15,
        warming_level=GWL
    )
    print(f"Data retrieved for {variable} in {time.time() - ds_start_time:.2f} seconds.")
    calculate_and_plot(ds, variable, GWL, timescale, [10, 50, 90])

def get_multiple_data(variables_units):
    for variable_unit in variables_units:
        process_variable(variable_unit)

variables_units = [
    # ("Precipitation (total)", "inches","Statistical","monthly",[1.0,1.5,2.0]),
    # ("Maximum air temperature at 2m", "degF","Statistical","monthly",[1.0,1.5,2.0]),
    # ("Minimum air temperature at 2m", "degF","Statistical","monthly",[1.0,1.5,2.0]),
    ("Maximum wind speed at 10m","m/s","Dynamical","monthly",[1.0,1.5,2.0]),
    # ("Fosberg fire weather index","[0 to 100]","Dynamical","hourly",[2.0])
]

get_multiple_data(variables_units)

Processing variable: Precipitation (total)
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

WARNING FOR WARMING LEVELS APPROACH
-----------------------------------
There may be NaNs in your data for certain simulation/warming level combinations if the warming level is not reached for that particular simulation before the year 2100. 

This does not mean you have missing data, but rather a feature of how the data is combined in retrieval to return a single data object. 

If you want to remove these empty simulations, it is recommended to first subset the data object by each individual warming level and then dropping NaN values.
Data retrieved for Precipitation (total) in 78.02 seconds.
Starting calculations for Precipitation (total)...
<xarray